# Module 12

### Part 3


In [1]:
# import 
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

### Exploratory Analysis

#### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Write a match query to find the establishments with a hygiene score of 20

# Use count_documents to display the number of documents in the result
query = {'scores.Hygiene': 20}
hygiene_count = establishments.count_documents(query)
print(hygiene_count)

# Put the pipeline together
match_query = {'$match': {'scores.Hygiene': 20}}  # Match establishments with a Hygiene score of 20
fields = {'$project': {'_id': 0, 'BusinessName': 1, 'scores.Hygiene': 1}}  # Project only BusinessName and Hygiene score

# Put the pipeline together
pipeline = [match_query, fields]

# Execute the pipeline and convert results to a list
results = list(establishments.aggregate(pipeline))

# pprint the 1st 10 items
pprint(results[:10])

41
[{'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'Brenalwood', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'Melrose Hotel', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'Seaford Pizza', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'Golden Palace', 'scores': {'Hygiene': 20}},
 {'BusinessName': "Ashby's Butchers", 'scores': {'Hygiene': 20}},
 {'BusinessName': 'South Sea Express Cuisine', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'Golden Palace', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'The Tulip Tree', 'scores': {'Hygiene': 20}},
 {'BusinessName': 'F & S', 'scores': {'Hygiene': 20}}]


In [7]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(hygiene_df))

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

Rows in DataFrame:  41


,BusinessName,scores
0,The Chase Rest Home,{'Hygiene': 20}
1,Brenalwood,{'Hygiene': 20}
2,Melrose Hotel,{'Hygiene': 20}
3,Seaford Pizza,{'Hygiene': 20}
4,Golden Palace,{'Hygiene': 20}
5,Ashby's Butchers,{'Hygiene': 20}
6,South Sea Express Cuisine,{'Hygiene': 20}
7,Golden Palace,{'Hygiene': 20}
8,The Tulip Tree,{'Hygiene': 20}
9,F & S,{'Hygiene': 20}


#### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
#  query to find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
match_query = {
    '$match': {
        'RatingValue': {'$gte': 4}, 'LocalAuthorityName': {"$regex":'London'}
    }
}
fields = {
    '$project': {
        '_id': 0, 
        'BusinessName': 1,  
        'RatingValue': 1 
    }
}
# Use count_documents to display the number of documents in the result
query = {
    'RatingValue': {'$gte': 4},
    'LocalAuthorityName': {"$regex":'London'}
}

london_ratings_count = establishments.count_documents(query)
print(london_ratings_count)

# Execute the pipeline
pipeline = [match_query, fields]
results = list(establishments.aggregate(pipeline))

# pprint the 1st 10 items
pprint(results[0:10])

33
[{'BusinessName': "Charlie's", 'RatingValue': 4},
 {'BusinessName': 'Mv City Cruises Erasmus', 'RatingValue': 5},
 {'BusinessName': 'Benfleet Motor Yacht Club', 'RatingValue': 4},
 {'BusinessName': 'Coombs Catering t/a The Lock and Key', 'RatingValue': 5},
 {'BusinessName': 'Tilbury Seafarers Centre', 'RatingValue': 5},
 {'BusinessName': 'Mv Valulla', 'RatingValue': 5},
 {'BusinessName': 'Tereza Joanne', 'RatingValue': 5},
 {'BusinessName': 'Brick Lane Brews', 'RatingValue': 4},
 {'BusinessName': 'The Nuance Group (UK) Limited', 'RatingValue': 5},
 {'BusinessName': 'WH Smith', 'RatingValue': 5}]


In [9]:
# make the DataFrame
London_ratings_df = pd.DataFrame(results)

# print amount of rows
print("Rows in DataFrame: ", len(London_ratings_df))

# .head to display 
London_ratings_df.head(10)

Rows in DataFrame:  33


,BusinessName,RatingValue
0,Charlie's,4
1,Mv City Cruises Erasmus,5
2,Benfleet Motor Yacht Club,4
3,Coombs Catering t/a The Lock and Key,5
4,Tilbury Seafarers Centre,5
5,Mv Valulla,5
6,Tereza Joanne,5
7,Brick Lane Brews,4
8,The Nuance Group (UK) Limited,5
9,WH Smith,5


#### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [32]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# latitude and longitude for Penang Flavours
latitude = 51.490142
longitude = 0.08384
degree_search = 0.01

#  query
match_query = {
    'RatingValue': 5,
    'geocode.latitude': {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    'geocode.longitude': {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

#  sort by hygiene score
sort =  {'scores.Hygiene': -1}

# Define the limit for the number of results
limit = 5

# Execute the pipeline
pipeline = [
    {'$match': match_query},
    {'$sort': sort},
    {'$limit': limit}
]
results = list(establishments.aggregate(pipeline))

# Print results
for result in results:
    pprint(result)


{'AddressLine1': '101 Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Lucky Food & Wine',
 'BusinessType': 'Retailers - other',
 'BusinessTypeID': 4613,
 'ChangesByServerID': 0,
 'Distance': 4647.024793263386,
 'FHRSID': 695287,
 'LocalAuthorityBusinessID': 'PI/000182135',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1SB',
 'RatingDate': '2022-06-25T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('668ef78483b640a8e33bd32b'),
 'geocode': {'latitude': 51.4878934, 'longitude': 0.0910104},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/695287',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractD

In [31]:
# Convert result to Pandas DataFrame
# make the DataFrame
top_5_df = pd.DataFrame(results)

# drop unessesary fields
sorted_top_5 = top_5_df[['BusinessName', 'scores', 'geocode']]
sorted_top_5.head()

,BusinessName,scores,geocode
0,Lucky Food & Wine,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...","{'longitude': 0.0910104, 'latitude': 51.4878934}"
1,Fineway Cash & Carry,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...","{'longitude': 0.0909382, 'latitude': 51.4876089}"
2,Everest Stores Ltd,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...","{'longitude': 0.0903232, 'latitude': 51.4876718}"
3,Premier Express,"{'Hygiene': 5, 'Structural': 0, 'ConfidenceInM...","{'longitude': 0.0904552, 'latitude': 51.4877234}"
4,TIWA N TIWA African Restaurant Ltd,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...","{'longitude': 0.0927429, 'latitude': 51.4870351}"


#### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [33]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# make pipe
match_query = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {
        '_id': '$LocalAuthorityName', 
        'total_establishments': {'$sum': 1}  
    }},
    {'$sort': {'total_establishments': -1}}
]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(results[0:10])

[{'AddressLine1': '101 Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Lucky Food & Wine',
  'BusinessType': 'Retailers - other',
  'BusinessTypeID': 4613,
  'ChangesByServerID': 0,
  'Distance': 4647.024793263386,
  'FHRSID': 695287,
  'LocalAuthorityBusinessID': 'PI/000182135',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1SB',
  'RatingDate': '2022-06-25T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('668ef78483b640a8e33bd32b'),
  'geocode': {'latitude': 51.4878934, 'longitude': 0.0910104},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/695287',
             'rel': 'self'}],
  'meta': {'dataSource

In [37]:
# make the DataFrame
local_authority_df = pd.DataFrame(results)

# print amount of rows
print("Rows in DataFrame: ", len(local_authority_df))

# .head to display 
local_authority_df.head()

Rows in DataFrame:  5


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,668ef78483b640a8e33bd32b,695287,0,PI/000182135,Lucky Food & Wine,Retailers - other,4613,101 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0910104, 'latitude': 51.4878934}",,4647.024793,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
1,668ef78483b640a8e33bd317,694606,0,PI/000116584,Fineway Cash & Carry,Retailers - other,4613,112 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0909382, 'latitude': 51.4876089}",,4647.016995,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,668ef78583b640a8e33bd34e,695033,0,PI/000172986,Everest Stores Ltd,Retailers - other,4613,104 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0903232, 'latitude': 51.4876718}",,4647.041728,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,668ef78583b640a8e33bd346,695566,0,PI/000188497,Premier Express,Retailers - other,4613,102 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0904552, 'latitude': 51.4877234}",,4647.038807,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,668ef78483b640a8e33bd238,1069652,0,PI/000206841,TIWA N TIWA African Restaurant Ltd,Restaurant/Cafe/Canteen,1,152 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0927429, 'latitude': 51.4870351}",,4646.930146,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
